In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/my_bayes_opt/')
import bo_new
import pandas as pd
import numpy as np
from data_analysis import get_heart_bounds, correlation_coef, graph_3d, get_index
from graph import narrow,corrplot3axes,trend,nearest,plot_exploration, graph_cc_distribution, cube, gp_plot, gp_plot2,predicted_visited, init_gp_plot
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer

In [2]:
def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))

In [3]:
ecgs = pd.read_csv("new_simu-data/Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv("new_simu-data/Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
init = 5
steps = 10

In [5]:
ecgs.shape

(4000, 7212)

In [ ]:
bounds = get_heart_bounds(labels)
# Pick out a sample to use as a target
tidx = np.random.randint(0, labels.shape[0])
target, target_ecg = labels[tidx], ecgs[tidx]
# optimizer = optimize_point(labels,bounds)  
optimizer = bo_new.mybo(f=black_box,pbounds=bounds, real_set=labels)
gp,X,rs,predicted = optimizer.gpfit(init_points=init, n_iter=steps,  acq="ucb", kappa = 1)
# print(optimizer.predicted)
graph_cc_distribution(target_ecg,ecgs,labels)
init_gp_plot(init,gp,labels,X)
predicted_visited(init,target,target_ecg,labels,ecgs,X,predicted)

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
    1 | 00m00s |    0.23803 |    3.1185 |  -51.4843 |   42.5245 | 
    2 | 00m00s |    0.60026 |   30.9194 |  -92.7589 |  -35.7565 | 
    3 | 00m00s |    0.66610 |   20.5774 |  -88.6341 |  -11.6335 | 
    4 | 00m00s |    0.07310 |   26.7018 |  -94.2967 |    9.0581 | 
    5 | 00m00s |    0.68626 |   32.6903 |  -92.7321 |  -21.3882 | 
Bayesian Optimization
-----------------------------------------------------------------
 Step |   Time |      Value |         x |         y |         z | 
    6 | 00m03s |    0.37224 |   22.0918 |  -37.5188 |  -26.6018 | 
    7 | 00m01s |    0.47725 |  -44.1595 |  -75.7426 |  -34.0211 | 
    8 | 00m01s |    0.09708 |  -22.9732 |  -50.0049 |   23.3087 | 
    9 | 00m01s |    0.57340 |  -15.8001 | -106.2170 |  -30.4722 | 
   10 | 00m01s |    0.14260 |   22.8993 |  -30.4229 |   19.8667 | 
   11 | 00m01s |    0.86485

In [ ]:
predicted